# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1482338175.log


This non-commercial license of GraphLab Create for academic use is assigned to divyarvshnkr@gmail.com and will expire on September 05, 2017.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.475815     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.506792     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
coeffModelAll = model_all.coefficients

In [7]:
for i in xrange(18):
    if(coeffModelAll[i]['value']!=0):
        print (coeffModelAll[i]['name'])

(intercept)
bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

 bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [22]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [26]:
l1_penalty_values = np.logspace(1, 7, num=13)
rss = []
rssOnly = []
for l1_penalty in l1_penalty_values:
    modell = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                             l2_penalty=0., l1_penalty=l1_penalty)
    predictions = modell.predict(validation)
    err = predictions - validation['price']
    sqErr = err**2
    rssCurr = sqErr.sum()
#     print l1_penalty
#     print rssCurr
    rssOnly.append(rssCurr)
    rss.append((l1_penalty,rssCurr))
    

In [27]:
rss

[(10.0, 625766285142460.5),
 (31.622776601683793, 625766285362394.4),
 (100.0, 625766286057885.1),
 (316.22776601683796, 625766288257224.4),
 (1000.0, 625766295212186.1),
 (3162.2776601683795, 625766317206080.8),
 (10000.0, 625766386760658.0),
 (31622.776601683792, 625766606749278.5),
 (100000.0, 625767302791634.9),
 (316227.76601683791, 625769507643885.8),
 (1000000.0, 625776517727024.5),
 (3162277.6601683795, 625799062845466.6),
 (10000000.0, 625883719085425.0)]

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [28]:
rssArr = np.array(rssOnly)
minIndex = np.argmin(rssArr)
rssmin = np.min(rssOnly)
print minIndex
print ('rss min is %.2f'% rssmin)
l1_min = l1_penalty_values[minIndex]

0
rss min is 625766285142460.50


In [29]:
l1_min

10.0

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [30]:
modellBest = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                             l2_penalty=0., l1_penalty=l1_min)

In [31]:
print modellBest.coefficients['value'].nnz()

18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [32]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [33]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [34]:
l1_penalty_values = np.logspace(8, 10, num=20)
nnzArr = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                             l2_penalty=0., l1_penalty=l1_penalty)
    nzz = model['coefficients']['value'].nnz()
    nnzArr.append((l1_penalty,nzz))
    
print nnzArr

[(100000000.0, 18), (127427498.57031322, 18), (162377673.91887242, 18), (206913808.11147901, 18), (263665089.87303555, 17), (335981828.62837881, 17), (428133239.8719396, 17), (545559478.11685145, 17), (695192796.17755914, 17), (885866790.41008317, 16), (1128837891.6846883, 15), (1438449888.2876658, 15), (1832980710.8324375, 13), (2335721469.0901213, 12), (2976351441.6313133, 10), (3792690190.7322536, 6), (4832930238.5717525, 5), (6158482110.6602545, 3), (7847599703.5146227, 1), (10000000000.0, 1)]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [35]:
l1_penalty_min = 0
l1_penalty_max = 0
for l1_penaltyy in nnzArr:
    l1_pen = l1_penaltyy[0]
    nnz = l1_penaltyy[1]
    
    if(nnz > max_nonzeros):
        if(l1_penalty_min < l1_pen):
            l1_penalty_min = l1_pen
    else:
        if(l1_penalty_max > l1_pen or l1_penalty_max == 0):
            l1_penalty_max = l1_pen
print l1_penalty_min
print l1_penalty_max

l1_penalty_min_rounded = l1_penalty_min - (l1_penalty_min % 1000000)
l1_penalty_max_rounded = l1_penalty_max - (l1_penalty_max % 1000000)

print ('%d' % l1_penalty_min_rounded)
print ('%d' % l1_penalty_max_rounded)

2976351441.63
3792690190.73
2976000000
3792000000


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

### l1_penalty_min = 2976000000
### l1_penalty_max = 3793000000

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [36]:
l1_penalty_valuesNew = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [37]:
l1_penalty_valuesNew

array([  2.97635144e+09,   3.01931664e+09,   3.06228184e+09,
         3.10524703e+09,   3.14821223e+09,   3.19117743e+09,
         3.23414263e+09,   3.27710782e+09,   3.32007302e+09,
         3.36303822e+09,   3.40600341e+09,   3.44896861e+09,
         3.49193381e+09,   3.53489901e+09,   3.57786420e+09,
         3.62082940e+09,   3.66379460e+09,   3.70675980e+09,
         3.74972499e+09,   3.79269019e+09])

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [44]:
rssArrNarrow = []
rssOnlyNarrow = []
nnzArrNarrow = []
rssNarrow = []
rssCurr = 0
rssnnz = []

for l1_penalty in l1_penalty_valuesNew:
    model1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                             l2_penalty=0., l1_penalty=l1_penalty)
    predictions1 = model1.predict(testing)
    err1 = predictions1 - testing['price']
    sqErr1 = err1**2
    rssCurr = sqErr1.sum()
    
    rssOnlyNarrow.append(rssCurr)
    nzz = 0
    nzz = model1['coefficients']['value'].nnz()
    print ('%.2f - %.2f - %d' % (l1_penalty,rssCurr,nzz))
    nnzArrNarrow.append(nzz)
    rssNarrow.append((l1_penalty,rssCurr,nzz))
    if(nzz == max_nonzeros):
        rssnnz.append(rssCurr)

2976351441.63 - 225420664589747.34 - 10
3019316638.95 - 226934649057185.19 - 10
3062281836.27 - 228465182889293.31 - 10
3105247033.59 - 230218987896870.53 - 10
3148212230.92 - 232276343469903.00 - 10
3191177428.24 - 234387883630244.09 - 10
3234142625.56 - 236527783134369.75 - 10
3277107822.88 - 238696028978648.75 - 10
3320073020.20 - 240138282042653.72 - 8
3363038217.52 - 241070305495473.62 - 8
3406003414.84 - 242176285184400.94 - 8
3448968612.16 - 243048850729974.62 - 7
3491933809.48 - 244039423050441.81 - 7
3534899006.81 - 245177905213289.03 - 7
3577864204.13 - 246307289288321.00 - 7
3620829401.45 - 247460137168837.44 - 6
3663794598.77 - 248344040164226.56 - 6
3706759796.09 - 249293823239619.06 - 6
3749724993.41 - 250264013256388.06 - 6
3792690190.73 - 251258967249544.81 - 6


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [45]:
rssnnz

[243048850729974.62, 244039423050441.8, 245177905213289.03, 246307289288321.0]

In [46]:
rssOnlyArray = np.array(rssnnz)

In [48]:
np.min(rssOnlyArray)

243048850729974.62

In [49]:
minIndexNarrow = np.argmin(rssOnlyArray)

In [50]:
minIndexNarrow

0

In [51]:
l1_penalty_minmax = 3448968612.16
modelminmax = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                             l2_penalty=0., l1_penalty=l1_penalty_minmax)

In [52]:
coeffs = modelminmax.coefficients

In [53]:
for i in xrange(18):
    if(coeffs[i]['value']!=0):
        print (coeffs[i]['name'])
    

(intercept)
bedrooms
bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above


In [105]:
coeffs['name']

dtype: str
Rows: 18
['(intercept)', 'bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']